In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

In [2]:
def daily_K_line(event,n,open,close,high,low):
    # 日均事件K线
    T0open=event*open  # 以T0open作为基准
    pre_close = event * close.shift(axis = 1)
    data=pd.DataFrame(columns=['T'+str(j) for j in range(n)])
    for j in range(n):
        _open=(event*open.shift(-j,axis=1))/pre_close
        _close=(event*close.shift(-j,axis=1))/pre_close
        _low=(event*low.shift(-j,axis=1))/pre_close
        _high=(event*high.shift(-j,axis=1))/pre_close
        lst=[_open.mean().mean(),_close.mean().mean(),_low.mean().mean(),_high.mean().mean()]
        data['T'+str(j)]=lst
    data.index=['open','close','low','high']
    data=data.T
    data.index=pd.to_datetime(['20200101','20200102','20200103','20200104','20200105'])
    # mpf.plot(data,type='candle',style='default')
    return data

首次触板时间分组

In [3]:
def setg(x):
    if x<1:
        return 0
    elif x<2:
        return 1
    elif x<5:
        return 2
    elif x<15:
        return 3
    elif x<30:
        return 4
    elif x<60:
        return 5
    elif x<120:
        return 6
    elif x<180:
        return 7
    elif x<240:
        return 8

In [4]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
# open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
pre_close = ff.read('pre_close').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

In [5]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

In [6]:
def up_first(stock):
    first = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        start_1 = np.argwhere((data.loc[:,'high'] == daily_up_limit).values)[0][0]
        first.append(start_1)
    return first
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first,dict.keys()),total = len(dict.keys())))
up_first_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4414/4414 [03:55<00:00, 18.76it/s]


In [7]:
result = up_first_factor.copy().applymap(setg)

In [8]:
open_ = (ff.read('open') * ff.read('post')).loc[stock_index,time_index].copy()
close_ = (ff.read('close')* ff.read('post')).loc[stock_index,time_index].copy()
high_ = (ff.read('high')* ff.read('post')).loc[stock_index,time_index].copy()
low_ = (ff.read('low')* ff.read('post')).loc[stock_index,time_index].copy()

In [13]:
stock_shape = ['单次涨停','多次涨停','炸板股']
stat = pd.DataFrame(columns = ['封板形态','分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益'])
for shape in stock_shape:
    shape_data = ff.read(shape).copy() * ff.read('10cm')
    feature_data = result * ff.read('10cm')
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [shape,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1])]

In [14]:
stat.dropna().sort_values(by = 'T1日内收益',ascending = True)

,封板形态,分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益
18,炸板股,0,2.289559,2.0,0.767492,0.021726,-0.029151,-0.022862
19,炸板股,1,0.625404,0.0,0.421959,0.028971,-0.023399,-0.014321
26,炸板股,8,2.071044,1.0,0.723358,0.072762,-0.014956,-0.011768
20,炸板股,2,1.124865,1.0,0.617869,0.037585,-0.021452,-0.010309
21,炸板股,3,2.270183,2.0,0.808396,0.048158,-0.016937,-0.007218
25,炸板股,7,3.346609,2.0,0.824543,0.066322,-0.011157,-0.006284
22,炸板股,4,2.242196,2.0,0.787944,0.054736,-0.013720,-0.005667
23,炸板股,5,2.668461,2.0,0.832078,0.059525,-0.012286,-0.005334
17,多次涨停,8,1.005382,1.0,0.516685,0.100062,-0.004579,-0.004056
24,炸板股,6,3.266954,3.0,0.848224,0.063401,-0.011489,-0.002799


单次涨停 + 多次涨停 + 炸板股合并

In [15]:
stock_shape = ['单次涨停','多次涨停','炸板股']
stat = pd.DataFrame(columns = ['分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  (ff.read('单次涨停').copy().replace({np.nan:0}) + ff.read('多次涨停').copy().replace({np.nan:0}) + ff.read('炸板股').copy().replace({np.nan:0})).replace({0:np.nan}) * ff.read('10cm')
feature_data = result * ff.read('10cm')
for i in range(0,9):
    num_ = (feature_data == i).replace({False:np.nan,True:1.0})
    event_ =  shape_data * num_
    data = daily_K_line(event_,5,open_,close_,high_,low_)
    stat.loc[len(stat.index)] = [i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

In [16]:
stat.sort_values(by = '合计收益',ascending = True)

,分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
0,0.0,4.526372,4.0,0.889128,0.062234,0.006465,0.000722,-0.031301
8,8.0,6.226050,5.0,0.890205,0.091036,-0.001914,-0.000995,-0.010878
1,1.0,1.740581,2.0,0.747040,0.075552,0.016240,0.012243,-0.008207
5,5.0,7.566200,7.0,0.916039,0.084933,0.006987,0.009564,-0.008080
2,2.0,3.441335,3.0,0.870829,0.079408,0.013144,0.010413,-0.007448
4,4.0,6.503767,6.0,0.909580,0.083983,0.009079,0.007953,-0.006938
6,6.0,9.663079,9.0,0.912809,0.086511,0.006749,0.009253,-0.006740
7,7.0,9.894510,8.0,0.909580,0.088386,0.006264,0.008401,-0.005350
3,3.0,7.385361,7.0,0.907427,0.083311,0.011534,0.011058,-0.005155


开盘涨幅绝对值分组 + 首次触板时间分组

In [37]:
def retgroup(x):
    if x<0.92:
        return 0
    elif x<0.94:
        return 1
    elif x<0.96:
        return 2
    elif x<0.98:
        return 3
    elif x<1.00:
        return 4
    elif x<1.02:
        return 5
    elif x<1.04:
        return 6
    elif x<1.06:
        return 7
    elif x<1.08:
        return 8
    elif x<=1.106:
        return 9

In [38]:
ret_group = (ff.read('open').loc[stock_index,time_index].copy()/ff.read('pre_close').loc[stock_index,time_index].copy()).applymap(retgroup)

In [39]:
from tqdm import trange
stat = pd.DataFrame(columns = ['开盘涨幅分组','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  (ff.read('单次涨停').copy().replace({np.nan:0}) + ff.read('多次涨停').copy().replace({np.nan:0}) + ff.read('炸板股').copy().replace({np.nan:0})).replace({0:np.nan}) * (ff.read('10cm').copy())
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
for j in trange(10):
    ret_ = (ret_group == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_ * ret_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [05:08<00:00, 30.89s/it]


In [40]:
stat.sort_values(by = '合计收益',ascending = True).dropna().head(20)

,开盘涨幅分组,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
27,3.0,0.0,0.002153,0.0,0.002153,-0.016881,-0.022995,-0.041870,-0.139876
11,1.0,2.0,0.001076,0.0,0.001076,0.099138,-0.109914,-0.109914,-0.110776
18,2.0,0.0,0.001076,0.0,0.001076,0.025064,-0.007069,-0.006427,-0.082005
81,9.0,0.0,1.940797,1.0,0.715823,0.045790,-0.007686,-0.007815,-0.061896
36,4.0,0.0,0.043057,0.0,0.041981,0.039086,0.004521,-0.002369,-0.056393
15,1.0,6.0,0.012917,0.0,0.012917,0.067508,-0.015265,0.013936,-0.047757
7,0.0,7.0,0.026911,0.0,0.025834,0.084703,-0.027423,-0.001238,-0.042720
14,1.0,5.0,0.011841,0.0,0.011841,0.074662,-0.017162,0.022985,-0.042500
8,0.0,8.0,0.030140,0.0,0.027987,0.092623,-0.033048,-0.005064,-0.040425
17,1.0,8.0,0.038751,0.0,0.035522,0.087709,-0.024510,-0.026620,-0.036801


In [21]:
stat[stat['mean'] >= 0.2].sort_values(by = '合计收益',ascending = True).dropna()

,开盘涨幅分组,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
81,9.0,0.0,1.940797,1.0,0.715823,0.045790,-0.007686,-0.007815,-0.061896
35,3.0,8.0,0.413348,0.0,0.275565,0.088168,-0.010016,-0.008557,-0.021847
73,8.0,1.0,0.265877,0.0,0.219591,0.069142,0.010344,0.009413,-0.020514
72,8.0,0.0,1.030140,1.0,0.589882,0.071529,0.010213,0.001145,-0.018258
65,7.0,2.0,0.586652,0.0,0.420883,0.074448,0.008612,0.007539,-0.016940
55,6.0,1.0,0.382131,0.0,0.316469,0.072420,0.011539,0.004063,-0.016042
32,3.0,5.0,0.276642,0.0,0.218515,0.083309,0.001451,0.006239,-0.015240
74,8.0,2.0,0.263724,0.0,0.219591,0.070372,0.015126,0.017824,-0.014502
31,3.0,4.0,0.209903,0.0,0.176534,0.081101,0.004921,0.001511,-0.013978
33,3.0,6.0,0.369214,0.0,0.273412,0.083004,0.003373,0.004375,-0.013623


open0931ratio分组 + 首次触板时间分组

In [22]:
def five_quantile(x):
    result = x.copy()
    x[(result<=result.quantile(0.2))] = 0
    x[(result>result.quantile(0.2))*(result<=result.quantile(0.4))] = 1
    x[(result>result.quantile(0.4))*(result<=result.quantile(0.6))] = 2
    x[(result>result.quantile(0.6))*(result<=result.quantile(0.8))] = 3
    x[(result>result.quantile(0.8))] = 4
    return x

In [23]:
from tqdm import trange
stat = pd.DataFrame(columns = ['open0931ratio分组','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  (ff.read('单次涨停').copy().replace({np.nan:0}) + ff.read('多次涨停').copy().replace({np.nan:0}) + ff.read('炸板股').copy().replace({np.nan:0})).replace({0:np.nan}) * (ff.read('10cm').copy())
for j in trange(5):
    factor = ff.read('open0931ratio').copy().loc[:,time_index].apply(five_quantile)
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_ * factor_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:25<00:00, 41.18s/it]


In [24]:
stat.sort_values(by = '合计收益',ascending = True).dropna().head(20)

,open0931ratio分组,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
9,1.0,0.0,0.030140,0.0,0.025834,0.001370,-0.030382,-0.037959,-0.129012
0,0.0,0.0,0.024758,0.0,0.013994,-0.003109,-0.023648,-0.006365,-0.126757
27,3.0,0.0,0.057051,0.0,0.047363,0.011879,-0.031882,-0.018062,-0.120002
18,2.0,0.0,0.020452,0.0,0.018299,0.019205,-0.027105,-0.042523,-0.107900
1,0.0,1.0,0.023681,0.0,0.023681,0.040957,-0.013782,-0.014947,-0.072824
10,1.0,1.0,0.013994,0.0,0.013994,0.052159,-0.006214,-0.015214,-0.054055
19,2.0,1.0,0.021529,0.0,0.020452,0.068770,-0.005032,-0.020414,-0.036262
2,0.0,2.0,0.147470,0.0,0.133477,0.070194,-0.005035,-0.004960,-0.034842
28,3.0,1.0,0.054898,0.0,0.051668,0.070644,-0.002161,0.002126,-0.031517
36,4.0,0.0,4.393972,4.0,0.888052,0.063036,0.007120,0.001189,-0.029844


In [25]:
up.replace({1.0:np.nan,np.nan:1.0}) * up

,20200825,20200826,20200827,20200828,20200831,20200901,20200902,20200903,20200904,20200907,...,20240221,20240222,20240223,20240226,20240227,20240228,20240229,20240301,20240304,20240305
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873703.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873706.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873726.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873806.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


T0首板 + T1触板时间分组

In [26]:
from tqdm import trange
stat = pd.DataFrame(columns = ['是否首板','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
factor = up.shift(axis = 1).replace({1.0:0,np.nan:1.0}) 
for j in trange(2):
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  num_ * factor_ * open_up
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:08<00:00, 34.25s/it]


In [27]:
stat.sort_values(by = '合计收益',ascending = True)

,是否首板,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
8,0.0,8.0,0.674919,0.0,0.418730,0.087667,-0.010355,-0.010828,-0.022688
5,0.0,5.0,0.989236,1.0,0.559742,0.075750,0.002234,0.007859,-0.022016
4,0.0,4.0,1.050592,1.0,0.565124,0.073311,0.006342,0.007998,-0.020348
7,0.0,7.0,0.829925,0.0,0.482239,0.082333,0.000228,0.006385,-0.017439
2,0.0,2.0,1.096878,1.0,0.607104,0.071647,0.011919,0.007907,-0.016434
6,0.0,6.0,0.927879,1.0,0.517761,0.081594,0.002713,0.010593,-0.015693
1,0.0,1.0,0.818084,1.0,0.528525,0.070859,0.013943,0.011769,-0.015198
3,0.0,3.0,1.556512,1.0,0.705059,0.075897,0.009749,0.012166,-0.014355
0,0.0,0.0,1.657696,1.0,0.711518,0.072254,0.014578,0.004660,-0.013168
9,1.0,0.0,1.779333,1.0,0.769645,0.075411,0.015476,0.008322,-0.009113


In [28]:
from tqdm import trange
stat = pd.DataFrame(columns = ['是否首板','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
factor = (up.replace({np.nan:0}).rolling(window = 5,axis = 1,closed = 'left').sum() == 0) 
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for j in trange(2):
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  num_ * factor_ * open_up
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:09<00:00, 35.00s/it]


In [29]:
print('1：前五天没有出现涨停板 0：前五天种出现过涨停板')
stat.sort_values(by = '合计收益',ascending = True)

1：前五天没有出现涨停板 0：前五天种出现过涨停板


,是否首板,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
8,0.0,8.0,1.817008,1.0,0.693219,0.089483,-0.010688,-0.006942,-0.021205
2,0.0,2.0,1.564047,1.0,0.707212,0.075517,0.008575,0.007223,-0.015908
5,0.0,5.0,2.062433,2.0,0.775027,0.081881,0.004226,0.009260,-0.013892
7,0.0,7.0,2.277718,2.0,0.763186,0.085846,0.000684,0.006892,-0.013470
6,0.0,6.0,2.413348,2.0,0.783638,0.084143,0.002442,0.007917,-0.013415
1,0.0,1.0,1.037675,1.0,0.596340,0.072496,0.014316,0.012017,-0.013188
4,0.0,4.0,1.966631,2.0,0.751346,0.079416,0.007468,0.009220,-0.013116
0,0.0,0.0,1.949408,2.0,0.754575,0.073141,0.014793,0.005185,-0.012066
3,0.0,3.0,2.603875,2.0,0.821313,0.079485,0.010612,0.010769,-0.009903
9,1.0,0.0,1.487621,1.0,0.716900,0.074739,0.016257,0.008571,-0.009004


In [30]:
from tqdm import trange
stat = pd.DataFrame(columns = ['前10天内是否出现过涨停板','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
factor = (up.replace({np.nan:0}).rolling(window = 10,axis = 1,closed = 'left').sum() == 0) 
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for j in trange(2):
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  num_ * factor_ * open_up
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:10<00:00, 35.02s/it]


In [31]:
stat.sort_values(by = '合计收益',ascending = True)

,前10天内是否出现过涨停板,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
8,0.0,8.0,2.445640,2.0,0.766416,0.090289,-0.007848,-0.004745,-0.017559
2,0.0,2.0,1.880517,2.0,0.756728,0.076699,0.009917,0.009051,-0.013384
5,0.0,5.0,2.829925,2.0,0.821313,0.082587,0.004532,0.008167,-0.012881
1,0.0,1.0,1.150700,1.0,0.629709,0.073080,0.014438,0.012747,-0.012482
4,0.0,4.0,2.611410,2.0,0.809473,0.080527,0.007495,0.010404,-0.011979
0,0.0,0.0,2.100108,2.0,0.769645,0.073144,0.014976,0.005260,-0.011880
7,0.0,7.0,3.243272,3.0,0.827772,0.086433,0.001865,0.007100,-0.011702
6,0.0,6.0,3.443488,3.0,0.839612,0.084908,0.003975,0.008400,-0.011117
9,1.0,0.0,1.336921,1.0,0.681378,0.074612,0.016604,0.008887,-0.008784
3,0.0,3.0,3.276642,3.0,0.850377,0.081142,0.011253,0.011148,-0.007605


In [32]:
from tqdm import trange
stat = pd.DataFrame(columns = ['前30天内是否出现过涨停板','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
factor = (up.replace({np.nan:0}).rolling(window = 30,axis = 1,closed = 'left').sum() == 0) 
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for j in trange(2):
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  num_ * factor_ * open_up
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:59<00:00, 29.93s/it]


In [33]:
stat.sort_values(by = '合计收益',ascending = True)

,前30天内是否出现过涨停板,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
8,0.0,8.0,3.671690,3.0,0.839612,0.090719,-0.005403,-0.003408,-0.014683
2,0.0,2.0,2.348762,2.0,0.811625,0.077812,0.010533,0.009037,-0.011655
0,0.0,0.0,2.418730,2.0,0.799785,0.073620,0.014959,0.005445,-0.011421
1,0.0,1.0,1.292788,1.0,0.663079,0.074241,0.014955,0.013013,-0.010804
5,0.0,5.0,4.276642,4.0,0.869752,0.083752,0.005570,0.008922,-0.010678
4,0.0,4.0,3.770721,3.0,0.867600,0.082516,0.008220,0.009272,-0.009265
6,0.0,6.0,5.423036,5.0,0.883746,0.085948,0.004842,0.008432,-0.009210
7,0.0,7.0,5.199139,4.0,0.884822,0.087671,0.003571,0.006370,-0.008758
9,1.0,0.0,1.018299,1.0,0.599569,0.074519,0.017325,0.010347,-0.008156
13,1.0,4.0,2.733046,3.0,0.836383,0.084380,0.008969,0.007159,-0.006650


In [35]:
from tqdm import trange
stat = pd.DataFrame(columns = ['前60天内是否出现过涨停板','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
factor = (up.replace({np.nan:0}).rolling(window = 60,axis = 1,closed = 'left').sum() == 0) 
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for j in trange(2):
    factor_ = (factor == j).replace({False:np.nan,True:1.0})
    feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  num_ * factor_ * open_up
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
stat.sort_values(by = '合计收益',ascending = True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:58<00:00, 29.42s/it]


,前60天内是否出现过涨停板,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
8,0.0,8.0,4.476857,4.0,0.866523,0.090824,-0.004342,-0.002945,-0.013518
9,1.0,0.0,0.728741,0.0,0.469322,0.073653,0.014546,0.007430,-0.011801
0,0.0,0.0,2.708288,2.0,0.828848,0.073502,0.015425,0.006268,-0.011072
1,0.0,1.0,1.428418,1.0,0.692142,0.074690,0.014756,0.012078,-0.010554
5,0.0,5.0,5.280947,5.0,0.900969,0.084174,0.006249,0.009276,-0.009577
2,0.0,2.0,2.683531,2.0,0.836383,0.079091,0.011749,0.010038,-0.009160
6,0.0,6.0,6.745963,6.0,0.902045,0.086191,0.005540,0.008978,-0.008269
4,0.0,4.0,4.601722,4.0,0.896663,0.083469,0.008559,0.008728,-0.007972
7,0.0,7.0,6.573735,5.0,0.900969,0.087942,0.004752,0.007235,-0.007306
3,0.0,3.0,5.393972,5.0,0.898816,0.082827,0.010948,0.012070,-0.006225
